<a href="https://colab.research.google.com/github/LeoTortega/Suicide-Text-Classification-GPT/blob/main/Suicide_Text_Classification_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Gathering

Buscando dados que serão utilizados para treino e teste do nosso modelo de classificação de texto.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
# Carregando dataset com as mensagens que serão utilizadas para teste e treino
df = pd.read_csv("hf://datasets/Ram07/Detection-for-Suicide/detection_final_cleaned.csv")
df.drop('cleaned_text', axis=1, inplace=True)


,class,text
0,suicide,Ex Wife Threatening SuicideRecently I left my ...
1,non-suicide,Am I weird I don't get affected by compliments...
2,non-suicide,Finally 2020 is almost over... So I can never ...
3,suicide,i need helpjust help me im crying so hard
4,suicide,It ends tonight.I can’t do it anymore. \nI quit.


In [5]:
# Verificando as cinco primeiras linhas
df.head()

,class,text
0,suicide,Ex Wife Threatening SuicideRecently I left my ...
1,non-suicide,Am I weird I don't get affected by compliments...
2,non-suicide,Finally 2020 is almost over... So I can never ...
3,suicide,i need helpjust help me im crying so hard
4,suicide,It ends tonight.I can’t do it anymore. \nI quit.


# Limpeza dos dados
Nessa etapa limparemos a coluna "text" do nosso dataset. Aqui realizaremos a tokenização, identificaremos e limparemos as Stop Words que são desnecessárias e atrapalham o treinamento do modelo, e aplicaremos a lemantização.

In [ ]:
# Bibliotecas utilizadas para limpeza dos textos
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [8]:
# Tokenizer para dividir texto em palavras ou frases
nltk.download('punkt')
# Identifica e limpa as Stop Words do nosso dataset
nltk.download('stopwords')
# Aplica a lemantização
nltk.download('wordnet')

# Instanciando o lematizador
lemmatizer = WordNetLemmatizer()

# Carregar stop words em inglês
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
# Função para limpar o texto
def limpar_texto(texto):
  # Converter para letras minúsculas
  texto = texto.lower()

  # Remover caracteres especiais
  texto = re.sub(r'[^\w\s]', '', texto)

  # Remover números
  texto = re.sub(r'\d+', '', texto)

  # Tokenizar o texto
  tokens = word_tokenize(texto)

  # Remover Stop Words e lematizar
  tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

  # Reconstruir o texto limpo
  texto_limpo = ' '.join(tokens)

  return texto_limpo

In [16]:
df['cleaned_text'] = df['text'].apply(limpar_texto)
df.head()

,class,text,cleaned_text
0,suicide,Ex Wife Threatening SuicideRecently I left my ...,ex wife threatening suiciderecently left wife ...
1,non-suicide,Am I weird I don't get affected by compliments...,weird dont get affected compliment coming some...
2,non-suicide,Finally 2020 is almost over... So I can never ...,finally almost never hear bad year ever swear ...
3,suicide,i need helpjust help me im crying so hard,need helpjust help im cry hard
4,suicide,It ends tonight.I can’t do it anymore. \nI quit.,end tonighti cant anymore quit
